In [1]:
from connector import connect_to
import sqlalchemy as db
import pandas as pd

# Блок 1. Создание схемы и таблиц

## Описание задачи
1. В базе данных создайте новую схему adv_works.
2. На основе данных из файла создайте в схеме adv_works таблицы и соедините их между собою по основным и внешним ключам.

In [2]:
def create_schema(schema_name='adv_works'):
    connection = connect_to('postgres')
    with connection.begin():  
        connection.execute(db.text(f"CREATE SCHEMA IF NOT EXISTS {schema_name}"))
        print(f"Схема '{schema_name}' создана или уже существует.")
    connection.close()

create_schema()

Схема 'adv_works' создана или уже существует.


In [24]:
df1 = pd.read_excel('adventure_works.xlsx',sheet_name='Customers')
df2 = pd.read_excel('adventure_works.xlsx',sheet_name='Products')
df3 = pd.read_excel('adventure_works.xlsx',sheet_name='Territory')
df4 = pd.read_excel('adventure_works.xlsx',sheet_name='Sales')
df5 = pd.read_excel('adventure_works.xlsx',sheet_name='ProductCategory')
df6 = pd.read_excel('adventure_works.xlsx',sheet_name='ProductSubCategory')

In [ ]:
with connect_to('postgres') as conn:
            df6.to_sql(
            name='productsubcategory',
            con=conn,
            index=False,
            if_exists='append',
            schema='adv_works' 
        )

# Блок 2. Аналитические задачи

### Секция 1. Анализ клиентов

Задача 1:
Сегментация по доходу: Посчитайте средний годовой личный доход клиентов (YearlyIncome) в разбивке по роду деятельности (Occupation). Итоговая таблица должна содержать следующие поля: occupation, number_of_customers, avg_income.

In [5]:
with connect_to('postgres') as conn:
    df1=pd.read_sql("""
        select 
            occupation
            , count(*) as number_of_customers
            , round(avg(yearlyincome)::numeric, 2) as avg_income
        from adv_works.customers  
        group by occupation ;
""", conn)

Задача 2:
Семейный профиль: Посчитайте долю (в процентах) клиентов с детьми и долю клиентов без детей. Итоговая таблица должна содержать следующие поля: has_children (где 1 означает - имеет детей и 0 - не имеет детей), pct_of_customer_base.

In [8]:
with connect_to('postgres') as conn:
    df2=pd.read_sql("""
        select 
            case
                when numberchildrenathome > 0 then 1 
                else 0
            end as has_children
            , round(count(*) * 100.0 / sum(count(*)) over (), 2) as pct_of_customer_base
        from 
            adv_works.customers
        group by 
            case
                when numberchildrenathome > 0 then 1 
                else 0
            end
""", conn)

Задача 3:
Высокодоходные клиенты: Сформируйте список топ 10 клиентов с наибольшей суммой покупок (поле SalesAmount). Итоговая таблица должна содержать следующие поля: customer_key, customer_name, total_purchase.

In [11]:
with connect_to('postgres') as conn:
    df3=pd.read_sql("""
        select
            c.customerkey
            , c.name
            , round(sum(s.salesamount)::numeric, 2) as total_purchase
        from adv_works.customers c 
        left join adv_works.sales s on c.customerkey = s.customerkey
        group by c.customerkey
        order by total_purchase desc
        limit 10;
""", conn)

Задача 4:
Влияние семейного положения: Посчитайте среднюю сумму продаж в разбивке по семейному положению клиентов (MaritalStatus) и определите насколько сильно различаются средние суммы между двумя группами. Итоговая таблица должна содержать следующие поля: year, marital_status, avg_sales_amount.

In [4]:
with connect_to('postgres') as conn:
    df4=pd.read_sql("""
        select 
            extract(year from s.orderdate) as year
            , c.maritalstatus
            , round(avg(s.salesamount)::numeric, 2) as avg_sales_amount 
        from adv_works.sales s
        left join adv_works.customers c on s.customerkey = c.customerkey
        group by c.maritalstatus, year
        order by year, c.maritalstatus;
""", conn)

### Секция 2. Анализ продаж

Задача 1:
Ежемесячные продажи: Создайте отчёт продаж по месяцам за последние 2 года (2003, 2004). Итоговая таблица должна содержать следующие поля: year, monthkey, month_name, sales_count (количество продаж), sales_amount.

In [10]:
with connect_to('postgres') as conn:
    df5=pd.read_sql("""
        select 
            extract(year from s.orderdate) as year
            , extract(month from s.orderdate) as month_key
            , to_char(s.orderdate, 'month') as month_name
            , count(s.*) as sales_count
            , round(sum(s.salesamount)::numeric, 2) as sales_amount
        from adv_works.sales s
        where extract(year from s.orderdate) in (2003,2004)
        group by year, month_key, month_name
        order by year, month_key;
""", conn)

Задача 2:
Продажи по регионам: Посчитайте сумму продаж в разбивке по регионам. Итоговая таблица должна содержать следующие поля: region, sales_count, sales_amount

In [13]:
with connect_to('postgres') as conn:
    df6=pd.read_sql("""
        select 
            t.region 
            , count(s.*) as sales_count
            , round(sum(s.salesamount)::numeric, 2) as sales_amount 
        from adv_works.sales s
        left join adv_works.territory t on s.salesterritorykey = t.territorykey
        group by t.region;
""", conn)

### Секция 3. Анализ продуктов

Задача 1:
Доля продаж: Посчитайте какую долю от общих продаж составляет каждая категория продуктов. Итоговая таблица должна содержать следующие поля: year, product_key, product_category_key, english_product_category_name, sales_amount, pct_of_total_sales

In [16]:
with connect_to('postgres') as conn:
    df7=pd.read_sql("""
        select 
            extract(year from s.orderdate) as year
            , s.productkey
            , p3.productcategorykey
            , p3.englishproductcategoryname as english_product_category_name
            , round(sum(s.salesamount)::numeric, 2) as sales_amount
            , round((100 * sum(s.salesamount) / (select sum(s2.salesamount) from adv_works.sales s2))::numeric, 2) as pct_of_total_sales
        from adv_works.sales s 
        left join adv_works.products p on s.productkey = p.productkey
        left join adv_works.productsubcategory p2 on p.productsubcategorykey = p2.productsubcategorykey
        left join adv_works.productcategory p3 on p2.productcategorykey = p3.productcategorykey 
        group by p3.productcategorykey, s.productkey, year
        order by year, s.productkey
        ;
""", conn)

Задача 2:
Самые продаваемые продукты: Определите топ 5 продуктов с наибольшей суммой продаж. Итоговая таблица должна содержать следующие поля: product_key, product_name, english_product_category_name, sales_amount

In [18]:
with connect_to('postgres') as conn:
    df8=pd.read_sql("""
        select 
            s.productkey 
            , p.productname
            , p3.englishproductcategoryname 
            , round(sum(s.salesamount)::numeric, 2) as sales_amount 
        from adv_works.sales s 
        left join adv_works.products p on s.productkey = p.productkey
        left join adv_works.productsubcategory p2 on p.productsubcategorykey = p2.productsubcategorykey
        left join adv_works.productcategory p3 on p2.productcategorykey = p3.productcategorykey
        group by s.productkey, p.productname, p3.englishproductcategoryname	
        order by sales_amount desc
        limit 5;
""", conn)

Задача 3:
Маржа от продаж: Посчитайте разницу между суммой продаж (SalesAmount) за минусом себестоимости (TotalProductCost), налогов (*TaxAmt) и расходов на доставку (Freight) по каждому продукту в разбивке по годам и месяцам. Итоговая таблица должна содержать следующие поля: year, monthkey, month_name, product_key, product_name, sales_amount, total_product_cost, tax_amt, freight, margin, margin_pct (маржа как процент от суммы продаж).

In [3]:
with connect_to('postgres') as conn:
    df9=pd.read_sql("""
        select 
            extract(year from s.orderdate) as year
            , extract(month from s.orderdate) as month_key
            , to_char(s.orderdate, 'month') as month_name
            , s.productkey as product_key
            , p.productname as product_name
            , sum(s.salesamount) as sales_amount
            , sum(s.totalproductcost) as total_product_cost
            , sum(s.taxamt) as tax_amt
            , sum(s.freight) as freight
            , round((sum(s.salesamount) - sum(s.totalproductcost) - sum(s.taxamt) - sum(s.freight))::numeric, 2) as margin
            , round(((sum(s.salesamount) - sum(s.totalproductcost) - sum(s.taxamt) - sum(s.freight)) * 100 / sum(s.salesamount))::numeric, 2) as margin_pct
        from adv_works.sales s
        left join adv_works.products p on s.productkey = p.productkey
        group by product_key, product_name, year, month_key, month_name
        order by product_key, year, month_key, sales_amount;
""", conn)

### Секция 4. Анализ трендов

Задача 1:
Квартальный рост: Посчитайте сумму продаж за каждый квартал и их процентное изменение по топ 2 наиболее продаваемым категориям. Итоговая таблица должна содержать следующие поля: year, quarter_id, product_category_key, english_product_category_name, quarter_sales_amount, quarter_over_quarter_growth_pct.

In [8]:
with connect_to('postgres') as conn:
    df10=pd.read_sql("""
        with quarterly_sales as (
            select 
                extract(year from s.orderdate) as year,
                extract(quarter from s.orderdate) as quarter_id,
                pc.productcategorykey as product_category_key,
                pc.englishproductcategoryname as english_product_category_name,
                sum(s.salesamount) as quarter_sales_amount
            from 
                adv_works.sales s
            left join 
                adv_works.products p on s.productkey = p.productkey
            left join 
                adv_works.productsubcategory ps on p.productsubcategorykey = ps.productsubcategorykey
            left join 
                adv_works.productcategory pc on ps.productcategorykey = pc.productcategorykey
            group by 
                year, quarter_id, product_category_key, english_product_category_name
        ),
        top_categories as (
            select 
                product_category_key,
                english_product_category_name,
                sum(quarter_sales_amount) as total_sales
            from 
                quarterly_sales
            group by 
                product_category_key, english_product_category_name
            order by 
                total_sales desc
            limit 2
        ),
        quarterly_growth as (
            select 
                qs.year,
                qs.quarter_id,
                qs.product_category_key,
                qs.english_product_category_name,
                qs.quarter_sales_amount,
                lag(qs.quarter_sales_amount) over (
                    partition by qs.product_category_key
                    order by qs.year, qs.quarter_id
                ) as previous_quarter_sales
            from 
                quarterly_sales qs
            join 
                top_categories tc on qs.product_category_key = tc.product_category_key
        )
        select 
            year,
            quarter_id,
            product_category_key,
            english_product_category_name,
            quarter_sales_amount,
            round((((quarter_sales_amount - previous_quarter_sales) / previous_quarter_sales) * 100)::numeric, 2) as quarter_over_quarter_growth_pct
        from 
            quarterly_growth
        order by 
            year, quarter_id, product_category_key;
""", conn)

Задача 2:
Сравнение будних и выходных (суббота, воскресенье) дней: Посчитайте продажи в разбивке по годам и дням недели. Определите в какие дни в среднем сумма продаж больше. Определите является ли сумма продаж больше в будние или выходные дни. Итоговая таблица должна содержать следующие поля: year, day_name, is_weekend (где 1 означает выходной а 0 будний день), sales_amount.


In [14]:
with connect_to('postgres') as conn:
    df11=pd.read_sql("""
        select 
            extract(year from s.orderdate) as year,
            to_char(s.orderdate, 'Day') as day_name,
            case 
                when extract(dow from s.orderdate) in (6, 0) then 1 -- 6 = Saturday, 0 = Sunday
                else 0 
            end as is_weekend,
            sum(s.salesamount) as sales_amount
        from 
            adv_works.sales s
        group by 
            year,
            day_name,
            is_weekend
        order by 
            year, is_weekend desc, day_name;
""", conn)